## 협업 필터링: 사용자 리뷰 기반

In [5]:
import pandas as pd
import surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


ModuleNotFoundError: No module named 'surprise'

In [1]:
ratings = pd.read_csv('data/ratings_small.csv')
ratings

NameError: name 'pd' is not defined

In [3]:
ratings['rating'].min()

0.5

In [4]:
ratings['rating'].max()

5.0

In [5]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader=reader)
data

In [6]:
svd = SVD(random_state=0)

In [7]:
cross_validate(svd,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8912  0.8953  0.8987  0.8991  0.8936  0.8956  0.0030  
MAE (testset)     0.6881  0.6896  0.6898  0.6936  0.6877  0.6898  0.0021  
Fit time          0.71    0.73    0.78    0.75    0.74    0.74    0.02    
Test time         0.11    0.10    0.18    0.11    0.14    0.13    0.03    


{'test_rmse': array([0.8911797 , 0.89531458, 0.89866101, 0.89911716, 0.89355924]),
 'test_mae': array([0.68805227, 0.68959066, 0.68982825, 0.69355353, 0.68773193]),
 'fit_time': (0.713066577911377,
  0.7300479412078857,
  0.7849009037017822,
  0.747999906539917,
  0.7380275726318359),
 'test_time': (0.10571718215942383,
  0.10374689102172852,
  0.17852282524108887,
  0.1077125072479248,
  0.1406233310699463)}

In [8]:
train_set = data.build_full_trainset()
svd.fit(train_set)

In [9]:
# 1번 유저가 매긴 평점들
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [10]:
# 1번 유저가 302번 영화의 평점을 예측
svd.predict(1,302)

Prediction(uid=1, iid=302, r_ui=None, est=2.7142061734434044, details={'was_impossible': False})

In [11]:
# 1번 유저가 1029번 영화의 실제평점이 3점인데 
# 예측한 평점 : 2.88
svd.predict(1,1029,3)

Prediction(uid=1, iid=1029, r_ui=3, est=2.8814455446761933, details={'was_impossible': False})

In [12]:
# 봤던 영화 / 안본 영화 구분해서 데이터 추가 후 학습시키고
# 봤던 영화 평점을 ratings에 추가하고, 안본 영화 평점 전부 예측해서
# 그 중에서 평점이 가장 높은 영화 TOP10 추천